In [220]:
import re
import json
import pandas as pd
from collections import defaultdict

In [3]:
df_pic = pd.read_csv('data/category.csv')

In [4]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
import os
import openai
# openai.organization = "org-KoEVdZNtGdE2ogROhvn95jG7"
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.Model.list()

In [6]:
openai.api_key = ''

In [7]:
def get_response(message):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        temperature=0.7
    )
    
    return response

In [38]:
def cat2content(cat):
    message = f'''
    請以json格式列出"{cat}"的20項商品名稱
    如以下範例回傳 1:商品一, ...
    '''
    
    response = get_response(message)
    content = response.to_dict()['choices'][0]['message']['content']
    return content

In [39]:
# cat = '咖啡'
# message = f'''
#     請以json格式列出"{cat}"的20項商品名稱
#     如以下範例回傳 1:商品一, ...
#     '''

# content = cat2content('現做咖啡飲品')
# content

In [193]:
c = '精品／飾品'
cats[c] = cat2content(c)
cats[c]

'{\n    "1": "商品一",\n    "2": "商品二",\n    "3": "商品三",\n    "4": "商品四",\n    "5": "商品五",\n    "6": "商品六",\n    "7": "商品七",\n    "8": "商品八",\n    "9": "商品九",\n    "10": "商品十",\n    "11": "商品十一",\n    "12": "商品十二",\n    "13": "商品十三",\n    "14": "商品十四",\n    "15": "商品十五",\n    "16": "商品十六",\n    "17": "商品十七",\n    "18": "商品十八",\n    "19": "商品十九",\n    "20": "商品二十"\n}'

In [196]:
# # cats = dict()
# # for i, c in enumerate(df_pic.category):
# #     if c in cats:
# #         continue
# #     cats[c] = cat2content(c)
# #     print(i, c)
    
# import pickle
# with open('data/gen_sample_json.pkl', 'wb') as f:
#     pickle.dump(cats, f)

In [197]:
with open('data/gen_sample_json.pkl', 'rb') as f:
    test =  pickle.load(f)

In [198]:
def get(response):
    try:
        return json.loads(response)
    except:
        return json.loads(('{'+r.split('{')[1]).split('}')[0]+'}')

In [199]:
df_pic['response'] = df_pic.category.apply(cats.get)

In [200]:
for i, r in enumerate(df_pic['response']):
    get(r)

In [210]:
df_pic['gen_samples'] = df_pic.category.apply(cats.get).apply(get).apply(lambda x: list(x.values()))

In [225]:
coll = defaultdict(list)
for k, v in df_pic.iterrows():
    for s in v['gen_samples']:
        
        coll['item_type'].append(v['item_type'])
        coll['category'].append(v['category'])
        coll['sample'].append(s)
        
            

In [234]:
df_gen = pd.DataFrame.from_dict(coll)
df_gen = df_gen[df_gen['sample'].apply(lambda x: '商品' not in x)]

In [235]:
df_gen

,item_type,category,sample
0,咖啡類,現做咖啡飲品,美式咖啡
1,咖啡類,現做咖啡飲品,拿鐵咖啡
2,咖啡類,現做咖啡飲品,卡布奇諾
3,咖啡類,現做咖啡飲品,摩卡咖啡
4,咖啡類,現做咖啡飲品,焦糖瑪奇朵
...,...,...,...
3495,菸酒類,高粱,高粱麵包
3496,菸酒類,高粱,高粱麵粉
3497,菸酒類,高粱,高粱沙拉醬
3498,菸酒類,高粱,高粱醬油


In [236]:
df_gen.to_csv('data/df_gen.csv')